## <b> 데이터 시각화 </b>
#### 인지실험 데이터 시각화 관련 프로그램

In [2]:
import os
import pandas as pd
import numpy as np
import random
import math
import matplotlib
from  matplotlib import pyplot as plt
%matplotlib qt
import scipy
import librosa
import librosa.display
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import FVA.FVA as fva
from FVA.lpc import *
from FVA.detect import *
from FVA.mfcc import *
from FVA.animation import *
from FVAmanager import *
from DB.DBmanager import *
import matplotlib
from  matplotlib import pyplot as plt
%matplotlib qt
#-*-coding:utf-8-*-
import matplotlib.colors as mcolors
import matplotlib.ticker as mticker
matplotlib.rcParams['font.family'] ='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] =False

c:\Users\User\Desktop\2023전람회(음성)\프로그램\2023voice


## <b> Data Load </b>

In [3]:
nF1, nF2        =   10, 10
result, users   =   getDB()
formants        =   pd.read_excel("DB/sounds.xlsx")
vowels          =   {'ㅏ':0,'ㅓ':1,'ㅗ':2,'ㅜ':3,'ㅡ':4,'ㅣ':5,'ㅐ':6,'ㅔ':7}
colors          =   ['red','green','orange','dodgerblue','deeppink','aqua','blueviolet','yellow']
counts          =   np.zeros((8,nF1,nF2))
userDict        =   {a[0]:a[1] for a in users.values.tolist()}

In [4]:
def WhiteListexception(input,filter):
   if (input=='권덕윤2') and ('권덕윤' in filter):
      return True
   if (input=='권덕윤') and ('권덕윤2' in filter):
      return True

def countSetup(namefilter=None,timefilter=None,time_ascending=True,nData=None,assertionerror=True):
   global result
   """
   Args:
      namefilter (list): [names]
      timefilter (list): [start, end] with "2023-10-03" style
   """
   def toymd(x):
      return x.split('T')[0]
   counts = np.zeros((8,nF1,nF2))
   result = result.sort_values(by=['at'],ascending=time_ascending)
   i = 0
   for x in result.values.tolist():
      if (namefilter is None) or ((userDict.get(x[4],None) in namefilter) or WhiteListexception(userDict.get(x[4],None),namefilter)):
         if (timefilter is None) or (timefilter[0]<=toymd(x[3]) and timefilter[0]<=toymd(x[3])):
            counts[vowels[x[2]]][int(x[1].split('_')[0])-1][int(x[1].split('_')[1].split('.')[0])-1] += 1
            i += 1
            if nData is not None and i>=nData: break
   
   if assertionerror: assert not assertionerror or nData is None or i >= nData, f"number of data({i}) is less then nData({nData})"
   elif not (not assertionerror or nData is None or i >= nData): return -1
   return counts.transpose(1,2,0)

In [5]:
NameFilter = ['손기웅']
TimeFilter = None#['2023-09-27','2023-10-03']

In [6]:
for x in users['name']:
    board = countSetup(namefilter=[x],timefilter=TimeFilter,nData=None)
    print(x,":",board.sum(),sep=" ")
board = countSetup()


방희준 : 2000.0
변상영 : 2001.0
강동오 : 389.0
서지혁 : 1000.0
김현서 : 2000.0
지민준 : 276.0
최홍제 : 3.0
곽민욱 : 2000.0
이종인 : 2000.0
권덕윤2 : 2011.0
송승환 : 2140.0
노수빈 : 1302.0
손기웅 : 109.0
임준혁 : 600.0
양세빈 : 2000.0
김지은 : 2000.0
정도일 : 803.0
권덕윤 : 2011.0
최희웡 : 2110.0


In [14]:
users

,id,name,email,phone
0,1tRUN17nz8SNpraO34mm0xJ2VS42,방희준,banghj1110@gmail.com,1031443382
1,8VmwhSj0CZOP98FGTSgWVh1elMK2,변상영,hufsbog22@gmail.com,1052549221
2,95qAzR5d0SVmQv1dQKLg3amtOiC2,강동오,louie0530@naver.com,1037650591
3,Be6njfAEawW9rmwE1LQV5ZfnlcB2,서지혁,jihyuksur921@gmail.com,1027436000
4,Do457fhfRKex3AtI5dQ8O2U7WEJ3,김현서,victoria524@naver.com,1082447820
5,E9HwoEHK6iZ5wuMaFYxaFZaljVo2,지민준,ddddewang1@cbs-h.cbe.go.kr,1082568240
6,FUQ7bswv1RbNsxPCudHciCNAOZz2,최홍제,enc2586@gmail.com,1031757526
7,LYrFonkX4AVb0UQrELmpCFA72On2,곽민욱,rhkralsdnr0401@gmail.com,1092947595
8,QWudxVycl7NxgKUBRMziC4fyIFm1,이종인,jjongin07@gmail.com,1098241016
9,UFOcT2n4CgS4FDD3DH5Y8jJBho93,권덕윤2,duck05220@naver.com,1058611286


In [60]:
asdf = []
for i in range(users.shape[0]):
    row = users.iloc[i]
    asdf.append([row['name'],row['id'],max(0,2000-len(result[result['user']==row['id']]))])
    
index = [x for x in asdf if x[0] in ['권덕윤2']]
nindex = [x for x in asdf if x[0] in ['권덕윤']]
for i in range(len(index)):
    nindex[i][2] += index[i][2]
    del index[i]

for x in asdf:
    print(x)

['방희준', '1tRUN17nz8SNpraO34mm0xJ2VS42', 0]
['변상영', '8VmwhSj0CZOP98FGTSgWVh1elMK2', 0]
['강동오', '95qAzR5d0SVmQv1dQKLg3amtOiC2', 1611]
['서지혁', 'Be6njfAEawW9rmwE1LQV5ZfnlcB2', 1000]
['김현서', 'Do457fhfRKex3AtI5dQ8O2U7WEJ3', 0]
['지민준', 'E9HwoEHK6iZ5wuMaFYxaFZaljVo2', 1724]
['최홍제', 'FUQ7bswv1RbNsxPCudHciCNAOZz2', 1997]
['곽민욱', 'LYrFonkX4AVb0UQrELmpCFA72On2', 163]
['이종인', 'QWudxVycl7NxgKUBRMziC4fyIFm1', 0]
['권덕윤2', 'UFOcT2n4CgS4FDD3DH5Y8jJBho93', 1257]
['송승환', 'ZK04e1bLJsXgM7QUlOrHgIzTbFo1', 0]
['노수빈', 'dznwXCu4dMaMrSrKWoUy1QTLQht1', 700]
['손기웅', 'iwHTazaB0uUFQAqZRzJqchH2oDr1', 1895]
['임준혁', 'lFlcx6WlifOcfSjFZgdFP7hkVf82', 1400]
['양세빈', 'ueZ3qVI3SFOZ1nWx2Al4Fx0pv2d2', 0]
['김지은', 'uwgxF6VQmggSKpqimilzt8CCpM42', 0]
['정도일', 'uy7oxru7rrRZZjHJ29jYGrWAP7f1', 1197]
['권덕윤', 'yWAbgGiM1DcUw2sCCGIGkU1JguX2', 1989]
['최희웡', 'zPpgwetVy5WlW9B2XpDJbLDW7f82', 0]


## <b> Data Visuallization

In [6]:
k = 800

def softmax(x): # numpy array!!
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def probability(a): # numpy array!!
    sum = a.sum()
    return a/sum

def print_randDot(fig,xrange:list,yrange:list,color,prob:float):
    """
    Args:
        fig (figure): 
        xrange (list): [xmin, xmax]
        yrange (list): [ymin, ymax]
        color (str): color of dot (plt.scatter(color) style)
        prob (float): probability of dot in [0,1]
    """
    nDot = int(round(prob * k, 0))
    x = [random.uniform(yrange[0],yrange[1]) for i in range(nDot)]
    y = [random.uniform(xrange[0],xrange[1]) for i in range(nDot)]
    plt.scatter(x,y,c=color,marker='o',s=6,alpha=0.25)
    
fig = plt.figure()
ax  = fig.add_subplot()
plt.gca().set_aspect('equal', adjustable='box')
ax.set_ylim([0,nF1])
ax.set_xlim([0,nF2])
for i in range(nF1):
    for q in range(nF2):
        a = probability(np.array(board[i][q]))
        for w in range(8):
            print_randDot(fig,[9-i,10-i],[9-q,10-q],colors[w],a[w])

for i,vowel in enumerate(vowels.keys()):
    plt.scatter([],[],s=50,c=colors[i],marker='o',label=vowel)
plt.rc('legend', fontsize=20)
plt.legend(loc='lower left', bbox_to_anchor=(1.0,0.5))

plt.show()

In [70]:

for name in users['name']:
    print(name)
    NameFilter = [name]
    board = countSetup(namefilter=NameFilter,timefilter=TimeFilter)
    
    fig, ax = plt.subplots(nF1,nF2+1)
    plt.gca().set_aspect('equal', adjustable='box')

    axes = []
    labels = []
    for i,vowel in enumerate(vowels.keys()):
        print(vowel)
        axes.append(ax[0][0].scatter([],[],s=150,c=colors[i],marker='o'))
        labels.append(vowel)
    plt.legend(axes,labels,loc='lower left', bbox_to_anchor=(-22.2,3),fontsize=20)
    
    for i in range(nF1):
        for q in range(nF2+1):
            ax[i][nF2-q].xaxis.set_visible(False)
            ax[i][nF2-q].yaxis.set_visible(False)
            ax[i][nF2-q].axis('off')
            if q<nF2:
                if board[i][q].sum()!=0 and i + nF2 - q < 15: 
                    ax[i][nF2-q].pie(np.array(board[i][q]),colors=colors,radius=1.3)
                else:  ax[i][nF2-q].scatter([],[])
    if board[2,nF2//2].sum()!=0:
        ax[2,nF2//2].set_title(name,position=(-11,0),fontsize=24)
    else:
        ax[2,nF2//2].set_title(name,position=(-6.32,0),fontsize=24)
    plt.get_current_fig_manager().full_screen_toggle()
    plt.savefig('FVA\\results\\MVPresult_'+name)
    plt.show()

방희준
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
변상영
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
강동오
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
서지혁
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
김현서
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
지민준
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
최홍제
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
곽민욱
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
이종인
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
권덕윤2
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
송승환
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
노수빈
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
손기웅
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
임준혁
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
양세빈
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
김지은
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
정도일
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
권덕윤
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
최희웡
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ


In [22]:
n = 500
for name in users['name']:
    for w in range(2):
        
        print(name)
        NameFilter = [name]
        board = countSetup(namefilter=NameFilter,timefilter=TimeFilter,
                           time_ascending=not bool(w),nData=n,assertionerror=False)
        if type(board) == 'int' and board == -1:
            continue
        fig, ax = plt.subplots(nF1,nF2+1)
        plt.gca().set_aspect('equal', adjustable='box')

        axes = []
        labels = []
        for i,vowel in enumerate(vowels.keys()):
            print(vowel)
            axes.append(ax[0][0].scatter([],[],s=150,c=colors[i],marker='o'))
            labels.append(vowel)
        plt.legend(axes,labels,loc='lower left', bbox_to_anchor=(-22.2,3),fontsize=20)
        
        for i in range(nF1):
            for q in range(nF2+1):
                ax[i][nF2-q].xaxis.set_visible(False)
                ax[i][nF2-q].yaxis.set_visible(False)
                ax[i][nF2-q].axis('off')
                if q<nF2:
                    if board[i][q].sum()!=0 and i + nF2 - q < 15: 
                        ax[i][nF2-q].pie(np.array(board[i][q]),colors=colors,radius=1.3)
                    else:  ax[i][nF2-q].scatter([],[])
        if board[2,nF2//2].sum()!=0:
            ax[2,nF2//2].set_title(name+"\n"+(f"마지막 {n}개" if w else f"처음 {n}개"),position=(-11,0),fontsize=24)
        else:
            ax[2,nF2//2].set_title(name+"\n"+(f"마지막 {n}개" if w else f"처음 {n}개"),position=(-6.32,0),fontsize=24)
        plt.get_current_fig_manager().full_screen_toggle()
        plt.savefig('FVA\\results\\MVPresult_'+name+(f"_end_{n}" if w else f"_start_{n}"))
        plt.show()

방희준
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
방희준
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
변상영
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
변상영
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
강동오
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
강동오
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
서지혁
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
서지혁
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
김현서
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
김현서
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
지민준
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
지민준
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
최홍제
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
최홍제
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
곽민욱
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
곽민욱
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
이종인
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
이종인
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
권덕윤2
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
권덕윤2
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
송승환
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
송승환
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
노수빈
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
노수빈
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
손기웅
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
손기웅
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
임준혁
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
임준혁
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
양세빈
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
양세빈
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
김지은
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
김지은
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
정도일
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
정도일
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
권덕윤
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
권덕윤
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
최희웡
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ
최희웡
ㅏ
ㅓ
ㅗ
ㅜ
ㅡ
ㅣ
ㅐ
ㅔ


In [9]:
result

,id,audio,answer,at,user
17310,hj21D8SDAEboYrAEjPBi,7_8.wav,ㅐ,2023-09-10T23:28:04.206Z,FUQ7bswv1RbNsxPCudHciCNAOZz2
2920,7Kdtk0KioW5ByGq3tSAw,2_5.wav,ㅡ,2023-09-11T16:44:18.733Z,dznwXCu4dMaMrSrKWoUy1QTLQht1
18860,lbx2raJx1lwehTcaNo0p,9_5.wav,ㅏ,2023-09-11T16:44:27.757Z,dznwXCu4dMaMrSrKWoUy1QTLQht1
4477,BGpzBj3IJlat7tkdEHzF,3_2.wav,ㅜ,2023-09-11T16:44:40.243Z,dznwXCu4dMaMrSrKWoUy1QTLQht1
4208,AY7FUsdqoGhw6DDUN7r1,1_3.wav,ㅜ,2023-09-11T16:44:45.199Z,dznwXCu4dMaMrSrKWoUy1QTLQht1
...,...,...,...,...,...
20209,p4NHztiDUDyByIb2Thmn,5_4.wav,ㅜ,2023-10-06T04:22:31.651Z,UFOcT2n4CgS4FDD3DH5Y8jJBho93
16012,eMoIGx6DtjnebZhrRGJq,7_9.wav,ㅔ,2023-10-06T04:22:39.751Z,yWAbgGiM1DcUw2sCCGIGkU1JguX2
16141,ekaUSyH77QQnpbF7HUWx,2_2.wav,ㅗ,2023-10-06T04:22:39.969Z,UFOcT2n4CgS4FDD3DH5Y8jJBho93
12163,USrVJIFH7USixW16kkcC,7_3.wav,ㅓ,2023-10-06T04:22:45.656Z,UFOcT2n4CgS4FDD3DH5Y8jJBho93


In [26]:
time_perception = []
result, users   =   getDB()
vowels          =   {'ㅏ':0,'ㅓ':1,'ㅗ':2,'ㅜ':3,'ㅡ':4,'ㅣ':5,'ㅐ':6,'ㅔ':7}
colors          =   ['red','green','orange','dodgerblue','deeppink','aqua','blueviolet','yellow']
counts          =   np.zeros((8,nF1,nF2))
userDict        =   {a[0]:a[1] for a in users.values.tolist()}
result          =   result.sort_values(by=['at'])

namefilter = ["권덕윤"]
timefilter = None

def WhiteListexception(input,filter):
   if (input=='권덕윤2') and ('권덕윤' in filter):
      return True
   if (input=='권덕윤') and ('권덕윤2' in filter):
      return True

def toymd(x):
    return x.split('T')[0]

for i,x in enumerate(result.values.tolist()):
    if (namefilter is None) \
    or ((userDict.get(x[4],None) in namefilter)\
    or WhiteListexception(userDict.get(x[4],None),namefilter)):
        if (timefilter is None) or (timefilter[0]<=toymd(x[3]) and timefilter[0]<=toymd(x[3])):
            time_perception.append(x)

print(time_perception)

[['I9VvohPLaQWI9l9wPgyX', '4_6.wav', 'ㅡ', '2023-09-26T12:23:16.382Z', 'yWAbgGiM1DcUw2sCCGIGkU1JguX2'], ['vXWszdz7EATfH6ezvdEF', '5_6.wav', 'ㅡ', '2023-09-26T12:23:26.599Z', 'yWAbgGiM1DcUw2sCCGIGkU1JguX2'], ['sUZoALg3OQicuCSAY3W4', '4_10.wav', 'ㅣ', '2023-09-26T12:23:36.186Z', 'yWAbgGiM1DcUw2sCCGIGkU1JguX2'], ['2Pp081JCEDBLG8fyELII', '7_6.wav', 'ㅏ', '2023-09-26T12:23:42.483Z', 'yWAbgGiM1DcUw2sCCGIGkU1JguX2'], ['vtKwo89ectCp7nZWcvDz', '5_4.wav', 'ㅗ', '2023-09-26T12:23:49.506Z', 'yWAbgGiM1DcUw2sCCGIGkU1JguX2'], ['80gKGKSVskRDqmOGYXse', '2_1.wav', 'ㅗ', '2023-09-26T12:23:59.120Z', 'yWAbgGiM1DcUw2sCCGIGkU1JguX2'], ['GFocHX1ryjx3eZzNDlDe', '7_6.wav', 'ㅏ', '2023-09-26T12:24:14.114Z', 'yWAbgGiM1DcUw2sCCGIGkU1JguX2'], ['5Uy7zE61tukKDh0HGsMT', '7_5.wav', 'ㅏ', '2023-09-26T12:24:26.099Z', 'yWAbgGiM1DcUw2sCCGIGkU1JguX2'], ['lfqKwcZCrqZdxuUfO7wb', '1_10.wav', 'ㅣ', '2023-09-26T12:24:32.192Z', 'yWAbgGiM1DcUw2sCCGIGkU1JguX2'], ['6BHMHBaFhIblaAgzxvtO', '4_4.wav', 'ㅜ', '2023-09-26T12:24:36.682Z', 'yWAbgGiM1

In [44]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
import numpy as np
plt.rcParams['animation.ffmpeg_path'] = 'C:\\ProgramData\\chocolatey\\bin\\ffmpeg.exe'

data    =   time_perception
time    =   4
nData   =   1000
board   =   np.zeros((10,10,8))
for i in range(nData):
    board[int(data[i][1].split('_')[0])-1][int(data[i][1].split('_')[1].split('.')[0])-1][vowels[data[i][2]]]+=1

fig, ax = plt.subplots(10,11)
plt.gca().set_aspect('equal',adjustable='box')

axes = []
labels = []
for i,vowel in enumerate(vowels.keys()):
    axes.append(ax[0][0].scatter([],[],s=150,c=colors[i],marker='o'))
    labels.append(vowel)
plt.legend(axes,labels,loc='lower left',bbox_to_anchor=(-22.2,3),fontsize=20)
for i in range(10):
    for q in range(11):
        ax[i][10-q].xaxis.set_visible(False)
        ax[i][10-q].yaxis.set_visible(False)
        ax[i][10-q].axis('off')
        if q<10:
            if board[i][q].sum()!=0 and i-q<5: 
                ax[i][10-q].pie(np.array(board[i][q]),colors=colors,radius=1.3)
            else: ax[i][10-q].scatter([],[])

def update(i):
    ny = int(data[i+nData+1][1].split('_')[0])-1
    nx = int(data[i+nData+1][1].split('_')[1].split('.')[0])-1
    board[ny][nx][vowels[data[i+nData+1][2]]] += 1
    ax[ny][10-nx].clear()
    ax[ny][10-nx].axis('off')
    ax[ny][10-nx].xaxis.set_visible(False)
    ax[ny][10-nx].yaxis.set_visible(False)
    if board[ny][nx].sum()!=0 and ny-nx<5:
        ax[ny][10-nx].pie(np.array(board[ny][nx]),colors=colors,radius=1.3)
    
    ny = int(data[i][1].split('_')[0])-1
    nx = int(data[i][1].split('_')[1].split('.')[0])-1
    board[ny][nx][vowels[data[i][2]]] -= 1
    ax[ny][10-nx].clear()
    ax[ny][10-nx].axis('off')
    ax[ny][10-nx].xaxis.set_visible(False)
    ax[ny][10-nx].yaxis.set_visible(False)
    if board[ny][nx].sum()!=0 and ny-nx<5:
        ax[ny][10-nx].pie(np.array(board[ny][nx]),colors=colors,radius=1.3)

ani = FuncAnimation(fig,update,frames=len(data)-nData-2,repeat=False,interval=1)

FFwriter = animation.FFMpegWriter(fps=200)
ani.save(r"C://Users/User/Desktop/animation.mp4",\
    writer=FFwriter,dpi=200)
    

IndexError: list index out of range

In [ ]:

print(name)
NameFilter = [name]
board = countSetup(namefilter=NameFilter,timefilter=TimeFilter,
                    time_ascending=not bool(w),nData=n,assertionerror=False)
if type(board) == 'int' and board == -1:
    continue
fig, ax = plt.subplots(nF1,nF2+1)
plt.gca().set_aspect('equal', adjustable='box')

axes = []
labels = []
for i,vowel in enumerate(vowels.keys()):
    print(vowel)
    axes.append(ax[0][0].scatter([],[],s=150,c=colors[i],marker='o'))
    labels.append(vowel)
plt.legend(axes,labels,loc='lower left', bbox_to_anchor=(-22.2,3),fontsize=20)

for i in range(nF1):
    for q in range(nF2+1):
        ax[i][nF2-q].xaxis.set_visible(False)
        ax[i][nF2-q].yaxis.set_visible(False)
        ax[i][nF2-q].axis('off')
        if q<nF2:
            if board[i][q].sum()!=0 and i + nF2 - q < 15: 
                ax[i][nF2-q].pie(np.array(board[i][q]),colors=colors,radius=1.3)
            else:  ax[i][nF2-q].scatter([],[])
if board[2,nF2//2].sum()!=0:
    ax[2,nF2//2].set_title(name+"\n"+(f"마지막 {n}개" if w else f"처음 {n}개"),position=(-11,0),fontsize=24)
else:
    ax[2,nF2//2].set_title(name+"\n"+(f"마지막 {n}개" if w else f"처음 {n}개"),position=(-6.32,0),fontsize=24)
plt.get_current_fig_manager().full_screen_toggle()
plt.savefig('FVA\\results\\MVPresult_'+name+(f"_end_{n}" if w else f"_start_{n}"))
plt.show()

In [17]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

colors = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue', 'limegreen', 
          'red', 'navy', 'blue', 'magenta', 'crimson']
explode = (0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, .01)
labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
nums = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

fig, ax = plt.subplots()

def update(num):
    print(num)
    ax.clear()
    ax.axis('equal')
    str_num = str(num)
    for x in range(10):
        nums[x] += str_num.count(str(x))
    ax.pie(nums, explode=explode, labels=labels, colors=colors, 
            autopct='%1.1f%%', shadow=True, startangle=140)
    ax.set_title(str_num)

ani = FuncAnimation(fig, update, frames=range(100), repeat=False)
plt.show()

0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
